# OpenSearch 클러스터 생성

#### 중요
* OpenSearch 클러스터 생성으로 인한 "과금" 이 발생이 되는 부분 유념 해주시기 바랍니다.
* SageMaker JupyterLab에서 아래 코드를 통해 OpenSearch Domain을 생성하는 경우, SageMaker Notebook IAM role에 <b>AmazonOpenSearchServiceFullAccess 권한</b>이 필요합니다.

## SageMaker Notebook 권한 설정

아래와 같이 SageMaker Notebook에서 사용하는 IAM role에 AmazonOpenSearchServiceFullAccess 권한을 추가해줍니다.
![image](https://github.com/hijigoo/aws-ebp-2024-2nd-round/assets/1788481/d7dd21cc-9bf6-474d-b33a-f0f3d016fc9d)

## OpenSearch 도메인 생성 (콘솔에서 진행하는 경우)

아래는 OpenSearch 콘솔 화면에서 UI로 도메인을 생성하는 절차 입니다. 그림 설명 다음의 코드를 통해서 도메인을 생성합니다.

#### Step1. OpenSearch 콘솔로 이동 후, Navigator에서 Domain 이동 후 Create domain 선택
![image](https://github.com/hijigoo/aws-ebp-2024-2nd-round/assets/1788481/c6901195-3312-42c3-9f9a-c0ce1db4f3ee)

#### Step2. Domain config 세팅
* Domain name :
* Domain creation Method: 사용자 지정생성 (손쉬운생성 선택시 '최대 절수' 오류 발생하는 경우)

![image](https://github.com/hijigoo/aws-ebp-2024-2nd-round/assets/1788481/db451836-8c90-4644-aaa4-d51701114a3a)

* Engine options: 2.11

* Network: Public access (실전에서는 VPC를 생성하여 VPC 액세스로 구성해야 합니다)

![image](https://github.com/hijigoo/aws-ebp-2024-2nd-round/assets/1788481/6d2ceeec-f1e6-48da-a97b-f90e5039167a)

* Master user: Create master user

* Master username, Master password and Confirm master password 입력

![image](https://github.com/hijigoo/aws-ebp-2024-2nd-round/assets/1788481/606f8542-5e42-45a8-95f6-d06c373c031c)

* 고급클러스터 > 최대절수 선택 (손쉬운 생성 오류 경우)

![image](https://github.com/hijigoo/aws-ebp-2024-2nd-round/assets/1788481/cc998588-dcf1-4ff9-8f04-9208c36d1209)

* 오른쪽 아래 주황색 create 선택

#### Step3. Access 설정
* 도메인 보안구성 > 편집 클릭

![image](https://github.com/hijigoo/aws-ebp-2024-2nd-round/assets/1788481/1649c9c3-3ef2-4085-8020-8227a0addb13)

* 도메인 수준 엑세스 정책 구성 > Effect : Allow 로 수정

![image](https://github.com/hijigoo/aws-ebp-2024-2nd-round/assets/1788481/804b0e75-585e-4263-ab62-5cf1e7ec9b70)


#### Step4. Domain enapoint 복사

![image](https://github.com/hijigoo/aws-ebp-2024-2nd-round/assets/1788481/049b27ff-1c9b-484b-a44e-68190723a8fd)

* [create_domain](https://boto3.amazonaws.com/v1/documentation/api/1.18.51/reference/services/opensearch.html#OpenSearchService.Client.create_domain)
* It takes about 20 mins

## OpenSearch 도메인 생성 (15~20분 소요)

In [2]:
import boto3
import uuid
import botocore
import time

DEV = True # True일 경우 1-AZ without standby로 생성, False일 경우 3-AZ with standby. 워크샵 목적일 때는 지나친 과금/리소스 방지를 위해 True로 설정하는 것을 권장
VERSION = "2.11" # OpenSearch Version (예: 2.7 / 2.9 / 2.11)

opensearch_user_id = 'raguser'
opensearch_user_password = 'Passw0rd1!'

region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity()["Account"]
opensearch = boto3.client('opensearch', region)
rand_str = uuid.uuid4().hex[:8]
domain_name = f'rag-hol-{rand_str}'

cluster_config_prod = {
    'InstanceCount': 3,
    'InstanceType': 'r6g.large.search',
    'ZoneAwarenessEnabled': True,
    'DedicatedMasterEnabled': True,
    'MultiAZWithStandbyEnabled': True,
    'DedicatedMasterType': 'r6g.large.search',
    'DedicatedMasterCount': 3
}

cluster_config_dev = {
    'InstanceCount': 1,
    'InstanceType': 'r6g.large.search',
    'ZoneAwarenessEnabled': False,
    'DedicatedMasterEnabled': False,
}


ebs_options = {
    'EBSEnabled': True,
    'VolumeType': 'gp3',
    'VolumeSize': 100,
}

advanced_security_options = {
    'Enabled': True,
    'InternalUserDatabaseEnabled': True,
    'MasterUserOptions': {
        'MasterUserName': opensearch_user_id,
        'MasterUserPassword': opensearch_user_password
    }
}

ap = f'{{\"Version\":\"2012-10-17\",\"Statement\":[{{\"Effect\":\"Allow\",\"Principal\":{{\"AWS\":\"*\"}},\"Action\":\"es:*\",\"Resource\":\"arn:aws:es:{region}:{account_id}:domain\/{domain_name}\/*\"}}]}}'

if DEV:
    cluster_config = cluster_config_dev
else:
    cluster_config = cluster_config_prod
    
response = opensearch.create_domain(
    DomainName=domain_name,
    EngineVersion=f'OpenSearch_{VERSION}',
    ClusterConfig=cluster_config,
    AccessPolicies=ap,
    EBSOptions=ebs_options,
    AdvancedSecurityOptions=advanced_security_options,
    NodeToNodeEncryptionOptions={'Enabled': True},
    EncryptionAtRestOptions={'Enabled': True},
    DomainEndpointOptions={'EnforceHTTPS': True}
)

KeyboardInterrupt: 

In [3]:
%%time
def wait_for_domain_creation(domain_name):
    try:
        response = opensearch.describe_domain(
            DomainName=domain_name
        )
        # Every 60 seconds, check whether the domain is processing.
        while 'Endpoint' not in response['DomainStatus']:
            print('Creating domain...')
            time.sleep(60)
            response = opensearch.describe_domain(
                DomainName=domain_name)

        # Once we exit the loop, the domain is ready for ingestion.
        endpoint = response['DomainStatus']['Endpoint']
        print('Domain endpoint ready to receive data: ' + endpoint)
    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'ResourceNotFoundException':
            print('Domain not found.')
        else:
            raise error

# OpenSearch 도메인 생성 - 약 20분 소요
wait_for_domain_creation(domain_name)

Domain not found.
CPU times: user 12.4 ms, sys: 2.46 ms, total: 14.9 ms
Wall time: 578 ms


In [4]:
response = opensearch.describe_domain(DomainName=domain_name)
opensearch_domain_endpoint = f"https://{response['DomainStatus']['Endpoint']}"

# OpenSearch 도메인 Endpoint 확인
print(opensearch_domain_endpoint)

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the DescribeDomain operation: Domain not found: rag-hol-2119dd9a

## (필요시) Clean-up : OpenSearch 도메인 삭제

비용 발생을 막기 위해 OpenSearch를 사용하지 않는다면 주석 제거 후 아래 코드를 실행하여 도메인을 삭제 합니다.<br>
domain_name은 변경해주셔야 합니다.<br>
(OpenSearch 콘솔에서 직접 생성한 도메인을 선택하고 Delete를 하셔도 됩니다)

In [ ]:
# import boto3
# import botocore

# opensearch = boto3.client('opensearch', region)


# # 삭제할 OpenSearch 도메인 이름을 콘솔에서 확인하고 입력합니다.
# domain_name = "rag-hol-f5833a6c"


# try:
#     # OpenSearch 도메인 삭제
#     response = opensearch.delete_domain(
#         DomainName=domain_name
#     )
#     print(f"Deleting domain '{domain_name}'...")

#     # 도메인이 완전히 삭제될 때까지 기다립니다.
#     while True:
#         try:
#             describe_response = opensearch.describe_domain(DomainName=domain_name)
#             status = describe_response['DomainStatus']['Processing']
#             if not status:
#                 print(f"Domain '{domain_name}' has been deleted.")
#                 break
#             else:
#                 print(f"Waiting for domain '{domain_name}' to be deleted...")
#                 time.sleep(60)
#         except botocore.exceptions.ClientError as e:
#             if e.response['Error']['Code'] == 'ResourceNotFoundException':
#                 print(f"Domain '{domain_name}' has been deleted.")
#                 break
#             else:
#                 raise e

# except botocore.exceptions.ClientError as e:
#     print(f"Error: {e.response['Error']['Message']}")
